In [ ]:
# ใช้อันนี้ และวนลูป
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.cross_decomposition import PLSRegression
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBRegressor
import random
import tensorflow as tf
import xgboost as xgb
import os
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error


# Initialize a DataFrame to store the results
results = pd.DataFrame(columns=["Classifier", "Input CSV", "R2 Train", "RMSE Train", 
                                 "Q2 CV", "RMSE CV", "Q2 Test", "RMSE Test"])

def randomforest_feature_selection(X, Y, num_features, output_file="feature_importances.csv"):
    """
    Select top `num_features` features using Random Forest Gini importance and compute mean Gini across different `mtry` values.
    
    Parameters:
    - X: Input features (DataFrame).
    - Y: Target values (Series or array-like).
    - num_features: Number of top features to select.
    - ind: List of different values of `mtry` (max features) to be used in the Random Forest.
    - output_file: File path to save the feature importances (default: 'feature_importances.csv').

    Returns:
    - X_selected: DataFrame with the top `num_features` selected.
    - top_features: List of top feature names.
    - mean_gini: List of mean Gini importances for each feature.
    """
    # Initialize a matrix to store the Gini importances for each model
    #ind = np.arange(1, X.shape[1] + 1)  # For example, using all features
    ind = np.array([2, 3, 5, 7, 9, 11, 13, 15, 17, 20])
    gini = np.zeros((X.shape[1], len(ind)))
    
    
    

    # Loop through each value of mtry in ind
    for i, mtry in enumerate(ind):
        rf = RandomForestRegressor(
            n_estimators=500, 
            max_features=mtry,  # Set the max features (mtry)
            random_state=42, 
            n_jobs=-1
        )
        
        # Fit the model to the data
        rf.fit(X, Y)
        
        # Store the feature importances based on Gini
        gini[:, i] = rf.feature_importances_

    # Compute the mean Gini for each feature across all models
    mean_gini = np.mean(gini, axis=1)

    # Create a DataFrame for feature importances
    feature_importances = pd.DataFrame(
        {'Feature': X.columns, 'Mean Gini Importance': mean_gini}
    ).sort_values(by='Mean Gini Importance', ascending=False)
    
    # Save feature importances to a CSV file
    feature_importances.to_csv(output_file, index=False)
    print(f"Feature importances saved to {output_file}")
    
    # Select top features based on mean Gini importance
    top_features = feature_importances.head(num_features)['Feature'].tolist()
    X_selected = X[top_features]
    
    print(f"Selected {len(top_features)} features using Random Forest (Mean Gini-based).")
    return X_selected, top_features

def build_model(X_internal, Y_internal, X_external, Y_external, seed, hx, input_csv_name):
    # Models to evaluate

    models = {
        "SVMP": SVR(kernel='poly', degree=1, C=1.0, epsilon=0.1),
     }   

    global results  # Use the global results DataFrame
    for name, model in models.items():
        model.fit(X_internal, Y_internal)

        # Predictions
        train_pred = model.predict(X_internal)
        cv_pred = cross_val_predict(model, X_internal, Y_internal, cv=10, n_jobs=-1)
        test_pred = model.predict(X_external)

        # Calculate metrics
        r2_train = r2_score(Y_internal, train_pred)
        rmse_train = np.sqrt(mean_squared_error(Y_internal, train_pred))
        mae_train = mean_absolute_error(Y_internal, train_pred)
        q2_cv = r2_score(Y_internal, cv_pred)
        rmse_cv = np.sqrt(mean_squared_error(Y_internal, cv_pred))
        mae_cv = mean_absolute_error(Y_internal, cv_pred)
        q2_test = r2_score(Y_external, test_pred)
        rmse_test = np.sqrt(mean_squared_error(Y_external, test_pred))
        mae_test = mean_absolute_error(Y_external, test_pred)

        # Append results
        # Append results
        new_row = pd.DataFrame({
    "Classifier": [name],
    "Input CSV": [input_csv_name],
    "R2 Train": [r2_train],
    "RMSE Train": [rmse_train],
    "MAE Train": [mae_train],        # เพิ่ม MAE Train
    "Q2 CV": [q2_cv],
    "RMSE CV": [rmse_cv],
    "MAE CV": [mae_cv],              # เพิ่ม MAE CV
    "Q2 Test": [q2_test],
    "RMSE Test": [rmse_test],
    "MAE Test": [mae_test]           # เพิ่ม MAE Test
})
        results = pd.concat([results, new_row], ignore_index=True)

    return results

# Main script

# Paths to the CSV files
internal_csv_path = 'feat_train.csv'
external_csv_path = 'feat_test.csv'

# Read the CSV files
df_internal = pd.read_csv(internal_csv_path).iloc[:, :]  # Remove the first column
df_external = pd.read_csv(external_csv_path).iloc[:, :]  # Remove the first column

# Target variable
y = [5.526221165, 5.026872146, 4.333482019, 3.440248935, 5.657577319, 9, 3.118729496, 6.776504059, 5.841637508, 9.045757491, 3.886056648, 5.200659451, 7.096910013, 5.367542708, 8.301029996, 6.107905397, 4.077793723, 5.853871964, 5.716698771, 3.109746695, 5.397940009, 5.886056648, 4.037630664, 5.068542129, 8.15490196, 4.154715874, 5.814457845, 4.392544977, 3.915852867, 4.337242168, 6.142667504, 6.602059991, 7.920818754, 6.785156152, 8.88941029, 5.522878745, 3.37675071, 7.576918042, 4.04769199, 7.072167867, 9.229147988, 6.207608311, 6.173925197, 2.193820026, 4.19314197, 5.823908741, 4.484126156, 5.899629455, 6.560193789, 3.431563586, 3.782516056, 4.839999931, 5.795880017, 4.752026734, 3.948847478, 4.575118363, 3.970169981, 3.037228288, 4.987162775, 4.612610174, 7.346787486, 5.008773924, 8.068542129, 5.07494588, 4.898940645, 7.318758763, 4.863279433, 3.291664097, 7.249029016, 6.698970004, 6.744727495, 7.30980392, 6.835052627, 6.065501549, 4.079876674, 5.004364805, 6.823908741, 4.853871964, 6.30980392, 5.59345982, 3.653647026, 6.522878745, 7.507239611, 6.239577517, 5.060480747, 4.302770657, 6.229147988, 5.779999191, 5.698970004, 2.568636236, 5.665546249, 4.631397041, 4.647817482, 4.920818754, 5.684029655, 5.790484985, 5.519993057, 2.397940009, 6.823908741, 6.45432185, 3.74052558, 6.961777362, 4.104577454, 7.366531544, 6.638272164, 6.48148606, 8.175223538, 4.406713933, 6.552841969, 4.801342913, 5.801342913, 4.518557371, 4.431798276, 5.552841969, 8.431798276, 3.721246399, 3.529883647, 7.301029996, 5.785818691, 7.381951903, 6.026872146, 4.886056648, 5.397940009, 4.194159451, 5.716698771, 7.886056648, 6.057991947, 3.84575867, 4.657577319, 5.919734373, 5.096910013, 6.026872146, 5.233587153, 6.244125144, 2.214670165, 4.061130178, 8, 4.659754238, 5.274088368, 4.677780705, 8.063989204, 5.886056648, 3.886056648, 6.387216143, 3.406381692, 8.008773924, 5.695007824, 8.568636236, 5.193820026, 4.392544977, 4.774690718, 6.004364805, 6.853871964, 5.54515514, 3.983509868, 4.066006836, 9.301029996, 10.22184875, 2.397940009, 3.178486472, 8.388276692, 8.677780705, 6.823908741, 7.920818754, 6.301029996, 5.468521083, 4.303556237, 5.844663963, 7.455931956, 5.056505484, 4.721246399, 3.962573502, 6, 3.060480747, 5.124938737, 9.022276395, 4.354577731, 5.455931956, 6.239999953, 4.382999659, 3.974694135, 8.15490196, 5.913640169, 3.162171756, 5.498940738, 6.301029996, 4.721246399, 4.387216143, 6.091514981, 7.638272164, 4.698970004, 6.698970004, 4.958607315, 5.806875402, 8.045757491, 4.869666232, 6.744727495, 6.508638306, 5.061980903, 4.251811973, 5.214670165, 7.920818754, 5.070581074, 5.180456064, 6.48148606, 4.772113295, 6.638272164, 5.008773924, 4.494850022, 2.301029996, 5.096910013, 7.823908741, 4.33999999, 5.147520006, 3.921688213, 6.095825632, 7.657577319, 4.526221165, 4.78968148, 2.585026652, 5.709520187, 4.096910013, 6.573537539, 4.562249437, 5.14327111, 5.496713336, 8.301029996, 4.527389802, 8.716698771, 6.022276395, 3.434305628, 6.387216143, 5.721246399, 5.193820026, 4.080398976, 5.95939766, 9, 8.166215625, 6.508638306, 5.239577517, 5.142667504, 6.970616222, 6.820448209, 5.749579998, 6.920818754, 4.001479117, 8.769551079, 7.886056648, 4.795880017, 7.27572413, 6.602059991, 3.520410295, 8.054531415, 3.904412253, 7.494850022, 3.119186408, 4.512296137, 7.7044329, 4.276134036, 4.886056648, 4.910094889, 5.193820026, 8.153972325, 6.958607315, 5.886056648, 4.721246399, 4.318668294, 6.331427731, 4.089375595, 4.753501419, 5.539102157, 3.151995729, 5.721475035, 4.235823868, 3.602059991, 5.886056648, 4.968187729, 7.15490196, 5.946921557, 4.200000031, 7.931814138, 6.301029996, 6.703839586, 5, 5.494850022, 4.050414849, 5.407823243, 4.477555766, 4.63638802, 8.619788758, 9.356547324, 9, 5.091514981, 5.583359493, 5.801342913, 5.991399828, 6.410497204, 3.530000004, 5.931814138, 6.309272456, 5.458420756, 5.661543506, 9.173925197, 6.638272164, 6.958607315, 1.431798276, 3.330000001, 8.522878745, 4.800793521, 5.94195377, 3.857110195, 6.593971055, 6.494850022, 6.026872146, 3.723538196, 5.298432015, 7.200659451, 7.379031564, 5.512861625, 6.698970004, 8.657577319, 5.698970004, 4.565431096, 4.970616222, 4.958607315, 5.494850022, 6.387216143, 4.467245621, 7.22184875, 6.494850022, 6.575118363, 7.537602002, 7.387216143, 5.527243551, 6.698970004, 6.537602002, 4.938924676, 4.876148359, 6.236572006, 4.698970004, 9.045757491, 7.37716452, 7.565590792, 5.050609993, 2.050609993, 3.391902054, 5.669586227, 6.48148606, 3.591336126, 8.173925197, 4.677780705, 2.420216403, 5.633763876, 6.001740662, 4.602059991, 5.30980392, 4.264800452, 4.155895769, 5, 6.509999358, 5.386052523, 7.397940009, 7.337242168, 6.08249449, 2.318758763, 4.337242168, 4.301029996, 7.508638306, 5.302509113, 7.283996656, 6.568636236, 7.602059991, 5.835647144, 3.684029655, 3.792688849, 4.703115524, 4.946921557, 6.677780705, 6.086186148, 4.300000029, 7.853871964, 9.004364805, 4.040000019, 7.568636236, 8.585026652, 8.327902142, 3.350000002, 3.844663963, 4.813608784, 4.950781977, 6.552841969, 5.379863945, 5.966576245, 3.783860499, 6.240332155, 6.677780705, 4.943095149, 9.15490196, 5.075720714, 4.602059991, 5.318758763, 6.77314243, 4.539102157, 8.625251654, 7.853871964, 4.879426069, 6.744727495, 8.301029996, 6.356547324, 10, 8.761953897, 4.657577319, 5.119186408, 7.494850022, 4.638272164, 6.337242168, 8.065501549, 5.37675071, 4.694648631, 5.740000205, 3.728158393, 5.639999555, 5.119186408, 5.638272164, 7.920818754, 4.769551079, 4.910094889, 3.869999994, 3.839831707, 4.351932871, 4.924453039, 5.66756154, 6.443697499, 8.096910013, 4.584692708, 7.721246399, 5.647817482, 6.022276395, 5.870000928, 7.958607315, 8.26760624, 8.397940009, 5.785156152, 4.142667504, 3.829999997, 6.292429824, 6.227238353, 4.610833916, 4.88941029, 8.387216143, 3.887830852, 4.09420412, 7.026872146, 4.732828272, 5.698970004, 5.589999287, 4.428291168, 6.017728767, 9.13667714, 8.337242168, 8.886056648, 7.301029996, 5.657577319, 8.096910013, 8, 8.725842151, 8.725842151, 3.966576245, 8, 8.48148606, 3.943095149, 8.104577454, 2.142667504, 5.7012124, 6.721246399, 8.537602002, 5.102372909, 5.66756154, 9.455931956, 7.619788758, 6.142667504, 7.995678626, 5.91721463, 4.785156152, 7.397940009, 5.638272164, 5.896196279, 5.975001527, 8.522878745, 7.528708289, 10.15490196, 3.987162775, 3.900664722, 7, 6.013228266, 6.853871964, 6.187086643, 6.187086643, 5.823908741, 5.055517328, 4.251811973, 8.698970004, 6.4867824, 8.698970004, 6.585026652, 9.301029996, 6.040958608, 5.036212173, 5.954677021, 9.455931956, 8.494850022, 7.924453039, 7.962573502, 4.287552343, 5.063486258, 5.480172006, 5.101274818, 6.107905397, 8.397940009, 5.454692884, 4.389999976, 5.752026734, 3.066644336, 5.232844134, 5.823908741, 4.721246399, 5.889999849, 10, 4.647817482, 3.987162775, 3.301000004, 7.15490196, 4.328827157, 6.517126416, 6.327902142, 8.045757491, 4.920818754, 6.568636236, 4.397940009, 6.22184875, 5.567030709, 5.652149144, 5.105407052, 5.812479279, 9.522878745, 5.372378149, 6.878112015, 5.346787486, 8.787812396, 6.375408541, 8.744727495, 7.315334136, 3.24131515, 8.096910013, 6.522878745, 5.798602876, 7.285837954, 4.860120914, 6.387216143, 5.215382707, 4.324221658, 5.619788758, 5.15490196, 5.779999191, 4.392544977, 4.463441557, 6.37675071, 4.654822383, 7.397940009, 5.392544977, 5.076755981, 8.721246399, 4.313900228, 8.657577319, 7.920818754, 8.119758224, 8.522878745, 7.562407968, 3.526075307, 5.522878745, 6.494850022, 5.345630909, 4.602059991, 8.42945706, 6.853871964, 8.795880017, 8.494850022, 7.15739076, 4.397940009, 5.293282218, 5.189095719, 7.585026652, 5.677780705, 7.677780705, 2.903089987, 4.090000009, 4.616184634, 6.236572006, 6.180456064, 5.677780705, 6.795880017, 8.769551079, 4.375717904, 5.89279003, 5.679999196, 3.602059991, 5.617982957, 4.943095149, 3.698970004, 6.698970004, 4.46470588, 7.054039296, 8.522878745, 5.070000019, 3.869666232, 4.26520017, 6.004364805, 8.638272164, 4.913640169, 7.229147988, 6.206999324, 8.522878745, 6.065501549, 7.920818754, 5.684029655, 6.880645119, 8.958607315, 6.055566682, 4.754487332, 5.431798276, 6.021531875, 4.518557371, 7.22184875, 5.330683119, 3.783781299, 4.52651303, 4.425968732, 3.04885684, 8.070581074, 6, 2.301029996, 6.173925197, 5.653647026, 5.501689446, 7.552841969, 5.447331784, 4.179142011, 5.608711951, 9.522878745, 8.657577319, 4.677780705, 7.698970004, 5.585026652, 5, 4.079999985, 4.510041521, 7.30980392, 4.906578315, 9.193820026, 4.166852888, 6.301029996, 4.164309429, 5.544698228, 7.807711387, 4.695294102, 3.958528359, 4.147032309, 7.230253533, 7.431798276, 7.841637508, 3, 5.863279433, 4.325138859, 6.13667714, 4.015697768, 4.172630727, 5.872895202, 7.443697499, 8.045757491, 4.795880017, 5.728158393, 7.22184875, 5.657577319, 9.113509275, 5.849999245, 8.698970004, 5.292429824, 4.197910742, 8.508638306, 4.476773958, 4.269999997, 4.742321425, 6.142667504, 8.522878745, 7.424812155, 3.304868702, 4.545765104, 4.621602099, 6.698970004, 7.744727495, 5.443697499, 7, 7.045757491, 5.058488567, 5.142667504, 5.585026652, 4.332920795, 6.366531544, 7.795880017, 5.870000928, 9.823908741, 3.985437462, 7.356547324, 3.171082438, 5.110138279, 5.886056648, 4.211124884, 5.723538196, 5.281498311, 8.15490196, 6.008773924, 5.59345982, 4.36552273, 8.301029996, 4.671620397, 7.709965389, 4.684029655, 8.301029996, 5.187086643, 3.583692413, 6.886056648, 6.647817482, 7.568636236, 4.818728228, 5.086186148, 4.294992041, 4.522878745, 7.677780705, 7.752026734, 4.160521953, 6.744727495, 7.782516056, 7.853871964, 7.698970004, 5.402304814, 5.327902142, 6.270373613, 4.562407968, 10.39794001, 5.798602876, 2.638272164, 3.071552937, 7.15490196, 8.744727495, 5.236572006, 5.102372909, 5.419075024, 4.325138859, 3.537691865, 6.853871964, 8.193820026, 9.15490196, 6.241088108, 2.301029996, 7.602059991, 3.548858579, 6.431798276, 4.585026652, 4.060480747, 4.221631657, 7.621057301, 6.552841969, 5.372634143, 6.647238808, 4.614393726, 8.15490196, 4.657577319, 7.886056648, 5.625251654, 5.943095149, 5.079354999, 7.522878745, 8.161150909, 4.327902142, 4.333107789, 6.709809516, 4.528708289, 5.889999849, 7.096910013, 6.13076828, 6.346787486, 4.721246399, 9.026872146, 3.724412259, 6.589999287, 9.229147988, 6.598944274, 7.778846678, 5.447331784, 3.886056648, 3.973303344, 6.744727495, 5.558304864, 7.958607315, 6.619788758, 5.116338565, 4.195179321, 9, 3.8569852, 4.739928612, 4.954677021, 4.935542011, 6.585026652, 3.91721463, 8.522878745, 5.369572125, 4.558304864, 6.642065153, 8.823908741, 5.1837587, 5.148741651, 5.568636236, 7.416801226, 7.962175249, 4.847711656, 7.769551079, 4.381951903, 4.669586227, 5.493494968, 5.555955204, 4.017728767, 6.823908741, 6.958607315, 8.004364805, 5.677780705, 7.920818754, 5.356547324, 5.406713933, 8.698970004, 4.43568909, 5.619788758, 6.301029996, 4.280088936, 7.657577319, 4.4867824, 6.420216403, 9.22184875, 5.537602002, 8.431798276, 6.27572413, 6.698970004, 4.651695137, 5.029653124, 3.716698771, 6.301029996, 4.288192771, 8.459670525, 4.238072162, 6.444905551, 3.286509457, 5.022733788, 4.835647144, 7.853871964, 5.384049948, 4.312203589, 9.522878745, 4.55129368, 5.124938737, 3.780520262, 8.640164518, 5.536107011, 5.399462706, 5.270592203, 5.036212173, 4.847711656, 6.541362151, 4.366531544, 8.060480747, 5.600326279, 9, 5.038389092, 5.522878745, 7.420216403, 4.525783736, 3.66756154, 4.017728767, 5.782516056, 9.301029996]
yt = [6.397940009, 5.888065724, 3.780000002, 5.749999856, 6.619788758, 7.823908741, 5.30980392, 3.886056648, 8.522878745, 5.684113584, 4.721246399, 4.48148606, 4.415668776, 4.047498152, 6.552841969, 2.552841969, 6.886056648, 7.397940009, 4.835647144, 5.412289035, 7.397940009, 7.920818754, 3.962573502, 7.342944147, 6.292429824, 7.406713933, 6.619788758, 5.972242795, 4.498940738, 5.033389013, 7.107905397, 5.795880017, 5.91829273, 3.407812135, 4.303643611, 6.036212173, 6.229147988, 6.102372909, 6.246416941, 5.330683119, 7.055517328, 5.707743929, 7.070581074, 6.27572413, 6.622875958, 5.188424994, 4.732828272, 7.494850022, 4.910094889, 3.994390555, 10, 5.619788758, 4.15490196, 4.508638306, 5.8569852, 7.823908741, 6.698970004, 6.193820026, 3.229999997, 3.369999999, 4.460422117, 6.638272164, 7.292429824, 6.022276395, 7.602059991, 4.48148606, 5.203425667, 7.100015437, 4.121018877, 8, 6.183096161, 8.045757491, 5.549750892, 8.096910013, 7.841637508, 7.698970004, 6.236572006, 7.397940009, 5.795880017, 4.492144128, 4.630784143, 5.005682847, 5.498940738, 8.744727495, 5.602059991, 5.026872146, 7.124938737, 8.15490196, 6.468521083, 5.698970004, 8.22184875, 3, 5.004364805, 5.417936637, 4.473660723, 3.912573543, 6.387216143, 8.522878745, 6.534617149, 5.698970004, 7.835647144, 3.560000001, 4.361910278, 7.187086643, 4.363512104, 9, 4.596879479, 5.93930216, 8.522878745, 4.204119983, 7.508638306, 9, 6.869666232, 4.243363892, 5.247183569, 4.036212173, 7, 9.045757491, 5.528708289, 4.112945622, 5.212539525, 7.283996656, 7.170696227, 8.537602002, 3.589999997, 6.576754126, 5.110698297, 4.335358024, 8.07109231, 6.677780705, 4.602059991, 3.879426069, 5.744727495, 4.492144128, 6.232102384, 2, 6.617982957, 4.886056648, 4.046723663, 9.301029996, 9.22184875, 2.283996656, 9.522878745, 6.698970004, 4.240105626, 7.625251654, 4.077326432, 7.384049948, 7.418961051, 4.651695137, 5.123205024, 7.563837353, 10.52287875, 8.045757491, 6.244918265, 5.301029996, 8.22184875, 4.598599459, 8.443697499, 3.829738285, 8.148741651, 8.638272164, 7.886056648, 4.004364805, 6.508638306, 6.019165216, 7.387216143, 7.795880017, 8.742321425, 4.345823458, 4.93330145, 7.958607315, 6.795880017, 5.397940009, 4.19239733, 5.359518563, 7.045757491, 8.096910013, 9.301029996, 8.657577319, 4.504455662, 5.607303047, 5.522878745, 5.57024772, 4.494850022, 6.853871964, 5.537602002, 4.20155654, 4.343422709, 5.334419009, 7, 5.052566278, 7.823908741, 4.247951552, 6.48148606, 9.283996656, 5.573488739, 3.839771643, 4.343326954, 4.876801925, 7.716698771, 5.362010219, 4.169999984, 6.013228266, 4.427128398, 6.638272164, 3.201487467, 6.036212173, 5.66756154, 3.797457195, 4.195179321, 3.978810701, 8.698970004, 8.537602002, 4.041961984, 3.102372909, 6.657577319, 4.958607315, 5.634512015, 6.327902142, 7.045757491, 6.443697499, 5.036212173]
# Separate features and target
X_internal = df_internal.drop(columns=['Target'], errors='ignore')
Y_internal = y #y[:len(df_internal)]
X_external = df_external.drop(columns=['Target'], errors='ignore')
Y_external = yt #y[:len(df_external)]

num_features_list = [50]

for num_features in num_features_list:

    if num_features == -1:
        X_internal_selected = X_internal  # Select all columns
        X_external_selected = X_external  # Select all columns

        #X_internal_selected, selected_features = randomforest_feature_selection(X_internal, Y_internal, num_features)
        #X_external_selected = X_external[selected_features]
    else:
        #X_internal_selected = X_internal.iloc[:, :num_features]
        #X_external_selected = X_external.iloc[:, :num_features]
        X_internal_selected, selected_features = randomforest_feature_selection(X_internal, Y_internal, num_features)
        X_external_selected = X_external[selected_features]

    # Train and evaluate models
    results = build_model(X_internal_selected, Y_internal, X_external_selected, Y_external, seed=67, hx=100, input_csv_name=f"Train_Test_{num_features}")
    
    print(f"Completed training for num_features = {num_features}")
results.to_csv('NAIResulsts.csv', index=False)
print("Model results saved.")
